# 🧊 Lab 3 — Optimal Policy for Frozen Lake
In this lab we will continue our exploration of Markov Decision Processes (MDPs) 
using the simplified **FrozenLake** environment. Building on Lab 2, we will 
practice three key ideas:

1. **Iterative Policy Evaluation** We compute the value of a given policy by repeatedly applying the Bellman expectation update until convergence, and compare this with the exact closed-form solution from Lab 2.

2. **Monte Carlo Simulation**  We estimate state values empirically by running many episodes in the FrozenLake    environment under the same policy, and compare these estimates to our analytical results.

3. **Finding the Optimal Policy (Value Iteration)**  We apply value iteration to compute the optimal state values and extract the corresponding optimal policy that maximizes long-term return.

In [1]:
import gymnasium as gym
import numpy as np
np.set_printoptions(precision=2, suppress=True)
import random
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [2]:
# Define a smaller 3x3 map
DESC_3x3 = [
    "SFF",
    "FHF",
    "FFG",
]
env = gym.make("FrozenLake-v1",desc=DESC_3x3,is_slippery=True, render_mode="ansi")
obs, info = env.reset(seed=42)
#print(env.render())

In [3]:
# General transition builder for FrozenLake-style maps (works for any rectangular map).
# Compatible with Gymnasium's action order: LEFT=0, DOWN=1, RIGHT=2, UP=3

from typing import List, Tuple
import numpy as np

LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3
ACTIONS = [LEFT, DOWN, RIGHT, UP]
DIRS = {
    LEFT:  (0, -1),
    DOWN:  (1, 0),
    RIGHT: (0, 1),
    UP:    (-1, 0),
}

def _grid_to_idx(r: int, c: int, ncols: int) -> int:
    return r * ncols + c

def _idx_to_grid(s: int, ncols: int) -> Tuple[int, int]:
    return divmod(s, ncols)

def _clip_move(r: int, c: int, dr: int, dc: int, nrows: int, ncols: int) -> Tuple[int, int]:
    rr, cc = r + dr, c + dc
    rr = min(max(rr, 0), nrows - 1)
    cc = min(max(cc, 0), ncols - 1)
    return rr, cc

def build_frozenlake_transitions(desc: List[str], is_slippery: bool = True):
    """
    Build transition probabilities for a FrozenLake-like grid.

    Args:
        desc: list of strings (rows), made of {'S','F','H','G'}.
        is_slippery: if True -> stochastic: {left, forward, right} each with prob 1/3.
                     if False -> deterministic in the intended direction.

    Returns:
        P: np.ndarray (S, A, S), transition probabilities
        R: np.ndarray (S, A, S), rewards (1 on entering 'G', else 0)
        absorbing: np.ndarray (S,), True for 'H' or 'G' (absorbing/self-loop)
        shape_2d: (nrows, ncols)
        flatten_map: np.ndarray (S,), identity map (r,c)->s ordering
    """
    nrows = len(desc)
    ncols = len(desc[0])
    S = nrows * ncols
    A = 4

    grid = np.array([list(row) for row in desc])
    is_hole = (grid == 'H')
    is_goal = (grid == 'G')
    absorbing = (is_hole | is_goal).reshape(-1)

    P = np.zeros((S, A, S), dtype=float)
    R = np.zeros((S, A, S), dtype=float)

    def step_from_state(s: int, a: int) -> int:
        r, c = _idx_to_grid(s, ncols)
        dr, dc = DIRS[a]
        rr, cc = _clip_move(r, c, dr, dc, nrows, ncols)
        return _grid_to_idx(rr, cc, ncols)

    for s in range(S):
        if absorbing[s]:
            # Absorbing states self-loop for all actions
            for a in ACTIONS:
                P[s, a, s] = 1.0
            continue

        for a in ACTIONS:
            if is_slippery:
                left = (a - 1) % 4
                right = (a + 1) % 4
                for aa in [left, a, right]:
                    s2 = step_from_state(s, aa)
                    P[s, a, s2] += 1.0/3.0
            else:
                s2 = step_from_state(s, a)
                P[s, a, s2] = 1.0

            # Reward for ARRIVING at goal
            for s2 in range(S):
                if P[s, a, s2] > 0:
                    rr, cc = _idx_to_grid(s2, ncols)
                    if grid[rr, cc] == 'G':
                        R[s, a, s2] = 1.0

    flatten_map = np.arange(S, dtype=int)
    return P, R, absorbing, (nrows, ncols), flatten_map

In [6]:
P, R, absorbing, shape2d, flatmap = build_frozenlake_transitions(DESC_3x3, is_slippery=True)

In [9]:
T_per_action = [P[:, a, :] for a in range(4)]  # list of 4 matrices, each (S,S)
P_all = np.array([T_per_action[0], T_per_action[1], T_per_action[2], T_per_action[3]])
print(P_all.shape)

(4, 9, 9)


In [13]:
POLICY = [
    1,  # state 0 (top-left)
    2,  # state 1
    1,  # state 2
    1,  # state 3
    1,  # state 4 (center, possibly hole)
    1,  # state 5
    2,  # state 6
    2,  # state 7
    2,  # state 8 (goal state)
]

n_states = len(POLICY)
P_pi = np.zeros((n_states, n_states))
for s in range(n_states):
    a = POLICY[s]             # action chosen at state s
    P_pi[s, :] = P_all[a, s]  # copy the probabilities for that action

Reward = [
0,  # state 0 (top-left)
0,  # state 1
0,  # state 2
0,  # state 3
0,  # state 4 (center, possibly hole)
0,  # state 5
0,  # state 6
0,  # state 7
1,  # state 8 (goal state)
]
Reward = np.array(Reward)
gamma = 0.9

## Part 1: Iterative Policy Evaluation

In Lab 2, we solved for the state-value function analytically using the 
closed-form expression:

$$
V = (I - \gamma P_\pi)^{-1} R
$$

In [15]:
# Solve for the value of the policy I designed 
I = np.eye(9)
A = I - gamma * P_pi
V = np.linalg.solve(A, Reward)
print("State values V:", V)

State values V: [0.3  0.36 0.83 0.36 0.   1.58 0.83 1.58 3.68]


While exact, this approach requires matrix inversion, which can be expensive 
for large state spaces. An alternative method is **iterative policy evaluation**, 
where we repeatedly apply the Bellman expectation update:

$$
V_{k+1}(s) \;=\; R(s) \;+\; \gamma \sum_{s'} P_\pi(s,s') \, V_k(s')
$$

### Steps
1. Initialize the value function arbitrarily (often $V_0 = R$ or $V_0 = 0$).  
2. Update all state values simultaneously using the Bellman update.  
3. Repeat for a number of iterations, or until values converge.  

In this exercise, we will:
- Start with $V_0 = R$.  
- Run the update for about 50 iterations.  
- Observe how the values converge to the same result as the closed-form 
  solution from Lab 2.  

In [16]:
# Your time to work on it
V_0 = Reward.copy()
for i in np.arange(50):
    V_0 = Reward + gamma*P_pi@V_0
print(V_0)

[0.3  0.36 0.83 0.36 0.   1.58 0.83 1.58 3.68]


## Part 2: Monte Carlo Simulation

In Part 1, we evaluated a fixed policy analytically and iteratively.  
Now we take a different approach: **simulation**.

The idea is to estimate the value of each state by running many episodes 
of the FrozenLake environment under the same policy, and then averaging 
the observed discounted returns.

Formally, the value of a state is defined as:

$$
V^\pi(s) = \mathbb{E}_\pi \Big[ \sum_{t=0}^{\infty} 
    \gamma^t R_{t+1} \;\Big|\; S_0 = s \Big]
$$

Monte Carlo methods approximate this expectation by repeated sampling:

1. Start from a given state $s$.  
2. Run an episode by following the policy $\pi$, recording the sequence 
   of rewards.  
3. Compute the discounted return $G = r_1 + \gamma r_2 + \gamma^2 r_3 + \dots$.  
4. Repeat many times and take the **average return** as an estimate of $V^\pi(s)$.  

### What to do
- Run many episodes (e.g., 5,000) under your chosen policy.  
- Collect the empirical state values.  
- Compare them with the results from **Part 1** (iterative evaluation) 
  and **Lab 2** (closed-form solution).  

In [7]:
# Your time to work on it
num_epi = 10000

DESC_3x3 = [
    "SFF",
    "FHF",
    "FFG",
]
env = gym.make("FrozenLake-v1",desc=DESC_3x3,is_slippery=True, render_mode="ansi")
gamma = 0.9


def run_epi(env, policy, start_state, render=False): 
    next_state, info = env.reset()
    '''
    Set the start state
    '''
    env.unwrapped.s = start_state 
    next_state = start_state
    
    current_gamma = 1/gamma
    if render:
        print(env.render())
    while True:
        current_gamma *= gamma 
        action = policy[next_state]
        next_state, reward, terminated, truncated, _ = env.step(action)
        if render:
            print(f"state:{next_state}, action: {action}, reward: {reward}")
            print(next_state, reward, terminated, truncated,)
            print(env.render())
        if terminated or truncated:
            if next_state == 8:
                epi_reward = current_gamma / (1-gamma)
            else:
                epi_reward = 0
            break
    return epi_reward

def run_state(state):
    total_reward = 0
    for i in np.arange(num_epi):
        reward = run_epi(env, POLICY, state)
        total_reward += reward
    return total_reward/num_epi

for i in np.arange(9):
    print(run_state(i)/3)  # I don't understand where did this /3 come from, let me know if you figure it out

0.3024339601879312
0.3577350783533311
0.8212359010718963
0.3657170622947161
0.0
1.5792289097629926
0.8396689337714301
1.604290363266661
3.333333333333334
